# **Cantoría Evaluation**

Developed by André Paiva

Based on Cantoría Dataset and papers by Helena Cuesta and Emilia Gómez

In [ ]:
import sys
import math
import numpy as np
import pandas as pd
import mir_eval.multipitch as mp
import sscs

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
SONG = 'CEA'
df_audio = pd.read_hdf('Datasets/Cantoria/Audio/Cantoria_' + SONG + '_Mix.h5', key='mix', mode='r')
df_pyin_s = pd.read_csv('Datasets/Cantoria/F0_pyin/Cantoria_' + SONG + '_S.csv', header=None, engine='pyarrow')
df_pyin_a = pd.read_csv('Datasets/Cantoria/F0_pyin/Cantoria_' + SONG + '_A.csv', header=None, engine='pyarrow')
df_pyin_t = pd.read_csv('Datasets/Cantoria/F0_pyin/Cantoria_' + SONG + '_T.csv', header=None, engine='pyarrow')
df_pyin_b = pd.read_csv('Datasets/Cantoria/F0_pyin/Cantoria_' + SONG + '_B.csv', header=None, engine='pyarrow')

mix = df_audio.to_numpy().T

#freq if freq >= 0 else 0

s_thres = 0.23
a_thres = 0.17
t_thres = 0.15
b_thres = 0.17

s_prob = np.array([prob >= s_thres for prob in df_pyin_s[2].to_numpy()[:mix.shape[1]]])
a_prob = np.array([prob >= a_thres for prob in df_pyin_a[2].to_numpy()[:mix.shape[1]]])
t_prob = np.array([prob >= t_thres for prob in df_pyin_t[2].to_numpy()[:mix.shape[1]]])
b_prob = np.array([prob >= b_thres for prob in df_pyin_b[2].to_numpy()[:mix.shape[1]]])

s_true = np.array([df_pyin_s[1].to_numpy()[i] if s_prob[i] else 32.7 for i in range(mix.shape[1])])
a_true = np.array([df_pyin_a[1].to_numpy()[i] if a_prob[i] else 32.7 for i in range(mix.shape[1])])
t_true = np.array([df_pyin_t[1].to_numpy()[i] if t_prob[i] else 32.7 for i in range(mix.shape[1])])
b_true = np.array([df_pyin_b[1].to_numpy()[i] if b_prob[i] else 32.7 for i in range(mix.shape[1])])

'''
s_true = np.array([freq/1000 if freq > 10000 else freq for freq in df_pyin_s[1].to_numpy()[:mix.shape[1]]])
a_true = np.array([freq/1000 if freq > 10000 else freq for freq in df_pyin_a[1].to_numpy()[:mix.shape[1]]])
t_true = np.array([freq/1000 if freq > 10000 else freq for freq in df_pyin_t[1].to_numpy()[:mix.shape[1]]])
b_true = np.array([freq/1000 if freq > 10000 else freq for freq in df_pyin_b[1].to_numpy()[:mix.shape[1]]])

s_true = np.array([32.7 if freq == 0 else freq for freq in s_true])
a_true = np.array([32.7 if freq == 0 else freq for freq in a_true])
t_true = np.array([32.7 if freq == 0 else freq for freq in t_true])
b_true = np.array([32.7 if freq == 0 else freq for freq in b_true])
'''

print(s_true.shape, mix.shape)

ckpt_dir = './Checkpoints/mask_voas_v2_treino1.keras'
model = sscs.mask_voas_cnn_v2_model()
sscs.load_weights(model, ckpt_dir=ckpt_dir)

splits = mix.shape[1]//256
splits_diff = 256 - (mix.shape[1] - splits * 256)
fill = np.zeros((360, splits_diff))
mix_filled = np.concatenate((np.copy(mix), fill), axis=1)
mix_filled = np.reshape(mix_filled, (360, -1, 256, 1)).transpose((1, 0, 2, 3))
batches = math.ceil(mix_filled.shape[0]/24)

s_pred_result = np.zeros((0, 360, 256))
a_pred_result = np.zeros((0, 360, 256))
t_pred_result = np.zeros((0, 360, 256))
b_pred_result = np.zeros((0, 360, 256))

for i in range(batches):
    s_pred, a_pred, t_pred, b_pred = model.predict(mix_filled[i*24:(i+1)*24])
    s_pred_result = np.append(s_pred_result, s_pred, axis=0)
    a_pred_result = np.append(a_pred_result, a_pred, axis=0)
    t_pred_result = np.append(t_pred_result, t_pred, axis=0)
    b_pred_result = np.append(b_pred_result, b_pred, axis=0)

s_pred_result = sscs.prediction_postproc(s_pred_result)[:, :mix.shape[1]]
a_pred_result = sscs.prediction_postproc(a_pred_result)[:, :mix.shape[1]]
t_pred_result = sscs.prediction_postproc(t_pred_result)[:, :mix.shape[1]]
b_pred_result = sscs.prediction_postproc(b_pred_result)[:, :mix.shape[1]]

s_f_score = sscs.f_score(s_true, s_pred_result, true_bin=False)
a_f_score = sscs.f_score(a_true, a_pred_result, true_bin=False)
t_f_score = sscs.f_score(t_true, t_pred_result, true_bin=False)
b_f_score = sscs.f_score(b_true, b_pred_result, true_bin=False)

print(s_f_score, a_f_score, t_f_score, b_f_score)

sscs.plot(mix)
sscs.plot(s_pred_result)
sscs.plot(a_pred_result)
sscs.plot(t_pred_result)
sscs.plot(b_pred_result)
sscs.plot(sscs.downsample_bins(s_pred_result.T).T)
sscs.plot(sscs.downsample_bins(a_pred_result.T).T)
sscs.plot(sscs.downsample_bins(t_pred_result.T).T)
sscs.plot(sscs.downsample_bins(b_pred_result.T).T)
sscs.song_to_midi(s_pred_result, a_pred_result, t_pred_result, b_pred_result, './MIDI/' + SONG + '.mid')